In [2]:
import geopandas as gpd

contiguous_usa = gpd.read_file('cb_2021_us_county_500k.shp')
contiguous_usa.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,20,161,00485044,0500000US20161,20161,Riley,Riley County,KS,Kansas,06,1579077672,32047392,"POLYGON ((-96.96095 39.28670, -96.96106 39.288..."
1,19,159,00465268,0500000US19159,19159,Ringgold,Ringgold County,IA,Iowa,06,1386932347,8723135,"POLYGON ((-94.47167 40.81255, -94.47166 40.819..."
2,30,009,01720111,0500000US30009,30009,Carbon,Carbon County,MT,Montana,06,5303728455,35213028,"POLYGON ((-109.79867 45.16734, -109.68779 45.1..."
3,16,007,00395090,0500000US16007,16007,Bear Lake,Bear Lake County,ID,Idaho,06,2527123155,191364281,"POLYGON ((-111.63452 42.57034, -111.63010 42.5..."
4,55,011,01581065,0500000US55011,55011,Buffalo,Buffalo County,WI,Wisconsin,06,1750290818,87549529,"POLYGON ((-92.08384 44.41200, -92.08310 44.414..."


In [13]:
import pandas as pd
county_pop = pd.read_excel('co-est2021-pop.xlsx')
county_pop.head()

,STATE_NAME,NAMELSAD,POPULATION
0,California,Alameda County,1648556
1,California,Alpine County,1235
2,California,Amador County,41259
3,California,Butte County,208309
4,California,Calaveras County,46221


In [14]:
#Merge shapefile with population data
pop_counties = contiguous_usa.merge(county_pop, left_on = 'NAMELSAD' , right_on =  'NAMELSAD')
pop_counties.head()

# Drop States
#pop_counties = pop_counties.loc[~pop_counties['STATE_NAME'].isin(['Alabama', 'Alaska', 'Arizona', 'Arkansa', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Georgia', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'])]

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME_x,LSAD,ALAND,AWATER,geometry,STATE_NAME_y,POPULATION
0,48,053,01383812,0500000US48053,48053,Burnet,Burnet County,TX,Texas,06,2576525880,69856136,"POLYGON ((-98.45924 31.01038, -98.45874 31.011...",Texas,50954
1,54,107,01560558,0500000US54107,54107,Wood,Wood County,WV,West Virginia,06,949229779,26590767,"POLYGON ((-81.75582 39.18052, -81.75575 39.180...",Texas,45875
2,48,499,01384035,0500000US48499,48499,Wood,Wood County,TX,Texas,06,1671172155,130731203,"POLYGON ((-95.66539 32.96043, -95.62536 32.960...",Texas,45875
3,55,141,01581130,0500000US55141,55141,Wood,Wood County,WI,Wisconsin,06,2053959984,41993013,"POLYGON ((-90.31752 44.29182, -90.31708 44.337...",Texas,45875
4,39,173,01074098,0500000US39173,39173,Wood,Wood County,OH,Ohio,06,1598441996,8718771,"POLYGON ((-83.88350 41.35712, -83.88334 41.384...",Texas,45875


In [15]:
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson = pop_counties.to_json())

In [16]:
# Define color palettes
palette = brewer['BuGn'][5]
palette = palette[::-1] # reverse order of colors so higher values have darker colors

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 10000000)

# Define custom tick labels for color bar.
tick_labels = {'0': '0', '100':'100',
               '1000':'1,000', '10,000': '10,000',
               '100000':'100,000', '1000000':'1,000,000',
               '10000000':'10,000,000+'}

# Create color bar.
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = 'horizontal',
                     major_label_overrides = tick_labels)

# Create figure object.
p = figure(title = '', 
           plot_height = 950,
           plot_width = 1900, 
           toolbar_location = 'below',
           tools = ('pan', 'wheel_zoom', 'box_zoom', 'reset')
          )
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Add patch renderer to figure.
states = p.patches('xs','ys', source = geosource,
                   fill_color = {'field':'POPULATION',
                                 'transform': color_mapper},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)

# Create hover tool
p.add_tools(HoverTool(renderers = [states],
                      tooltips = [('County','@NAMELSAD'),
                                ('Population','@POPULATION')]))

# Specify layout
p.add_layout(color_bar, 'below')

show(p)